In [1]:
import pandas as pd
import numpy as np
import re
from random import random
import emoji
from tqdm import notebook
def tqdm(x, **kargs):
    return notebook.tqdm(x, leave=False, **kargs)
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

In [2]:
from transformers import RobertaTokenizer, RobertaModel

I0622 05:56:37.220417 140500398008128 file_utils.py:39] PyTorch version 1.5.0 available.
/home/ino/anaconda3/envs/TrackNet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ino/anaconda3/envs/TrackNet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ino/anaconda3/envs/TrackNet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint1

## Loading data

In [3]:
df_train = pd.read_json('./source/train_gold.json', lines=True)
categories_type = pd.read_json('./source/categories.json', lines=True)
df_dev = pd.read_json('./source/dev_unlabeled.json', lines=True)
df_test = pd.read_json('./source/test_unlabeled.json', lines=True)
print("Number of text in training data: {}".format(df_train.shape[0]))
print("Number of text in categories: {}".format(categories_type.shape[1]))
print("Number of text in developing data: {}".format(df_dev.shape[0]))
print("Number of text in testing data: {}".format(df_test.shape[0]))

Number of text in training data: 32000
Number of text in categories: 43
Number of text in developing data: 4000
Number of text in testing data: 4000


In [4]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

I0622 05:56:49.801682 140500398008128 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /home/ino/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
I0622 05:56:49.803424 140500398008128 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt from cache at /home/ino/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
I0622 05:56:50.841392 140500398008128 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /home/ino/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b

In [9]:
# Output tokenizer vocab
tokenizer.save_vocabulary('./roberta_vocab/')

('./roberta_vocab/vocab.json', './roberta_vocab/merges.txt')

In [10]:
roberta_vocab = pd.read_json('roberta_vocab/vocab.json', typ='series')

In [11]:
list(roberta_vocab.keys())

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 '.',
 'Ġthe',
 ',',
 'Ġto',
 'Ġand',
 'Ġof',
 'Ġa',
 'Ġin',
 '-',
 'Ġfor',
 'Ġthat',
 'Ġon',
 'Ġis',
 'âĢ',
 "'s",
 'Ġwith',
 'ĠThe',
 'Ġwas',
 'Ġ"',
 'Ġat',
 'Ġit',
 'Ġas',
 'Ġsaid',
 'Ļ',
 'Ġbe',
 's',
 'Ġby',
 'Ġfrom',
 'Ġare',
 'Ġhave',
 'Ġhas',
 ':',
 'Ġ(',
 'Ġhe',
 'ĠI',
 'Ġhis',
 'Ġwill',
 'Ġan',
 'Ġthis',
 ')',
 'ĠâĢ',
 'Ġnot',
 'Ŀ',
 'Ġyou',
 'ľ',
 'Ġtheir',
 'Ġor',
 'Ġthey',
 'Ġwe',
 'Ġbut',
 'Ġwho',
 'Ġmore',
 'Ġhad',
 'Ġbeen',
 'Ġwere',
 'Ġabout',
 ',"',
 'Ġwhich',
 'Ġup',
 'Ġits',
 'Ġcan',
 'Ġone',
 'Ġout',
 'Ġalso',
 'Ġ$',
 'Ġher',
 'Ġall',
 'Ġafter',
 '."',
 '/',
 'Ġwould',
 "'t",
 'Ġyear',
 'Ġwhen',
 'Ġfirst',
 'Ġshe',
 'Ġtwo',
 'Ġover',
 'Ġpeople',
 'ĠA',
 'Ġour',
 'ĠIt',
 'Ġtime',
 'Ġthan',
 'Ġinto',
 'Ġthere',
 't',
 'ĠHe',
 'Ġnew',
 'ĠâĢĶ',
 'Ġlast',
 'Ġjust',
 'ĠIn',
 'Ġother',
 'Ġso',
 'Ġwhat',
 'I',
 'Ġlike',
 'a',
 'Ġsome',
 'S',
 'Ã«',
 'Ġthem',
 'Ġyears',
 "'",
 'Ġdo',
 'Ġyour',
 'Ġ-',
 'Ġ1',
 '"',
 'Ġif',
 'Ġcould',
 '?',

## Start analyze

In [14]:
def get_vocab(corpus):
    vocabulary = Counter()
    for sentance in corpus:
        for word in sentance.split():
            vocabulary.update([word])
    return vocabulary

In [15]:
def check_coverage(vocabs, roberta_vocab):
    known_words = {}
    unknown_words = {}
    known_count = 0
    unknown_count = 0
    for word in tqdm(vocabs.keys(), desc='Checking: '):
        if word in list(roberta_vocab.keys()):
            known_words[word] = roberta_vocab[word]
            known_count += vocabs[word]
        elif 'Ġ' + word in list(roberta_vocab.keys()):
            # If we have deep look in roberta tokenizer, many words combine with 'Ġ' since roberta is byte pair encoding
            known_words[word] = roberta_vocab['Ġ' + word]
            known_count += vocabs[word]            
        else:
            unknown_words[word] = vocabs[word]
            unknown_count += vocabs[word]
    print("Found embeddings for {:.3%} ({} / {}) of vocab".format(len(known_words) / len(vocabs), len(known_words), len(vocabs)))
    print("Found embeddings for {:.3%} ({} / {}) of all text".format(known_count / (known_count + unknown_count), known_count, (known_count + unknown_count)))
    return unknown_words

In [16]:
train_text_vocab = get_vocab(df_train['text'].values)
train_reply_vocab = get_vocab(df_train['reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
unknown_text = check_coverage(train_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 68961
train reply unique vocab count is: 25542


Found embeddings for 22.820% (15737 / 68961) of vocab
Found embeddings for 77.681% (508783 / 654963) of all text



Found embeddings for 30.307% (7741 / 25542) of vocab
Found embeddings for 73.164% (79306 / 108395) of all text


In [17]:
dev_text_vocab = get_vocab(df_dev['text'].values)
dev_reply_vocab = get_vocab(df_dev['reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 17684
dev reply unique vocab count is: 5360


Found embeddings for 39.064% (6908 / 17684) of vocab
Found embeddings for 77.472% (64280 / 82972) of all text



Found embeddings for 45.243% (2425 / 5360) of vocab
Found embeddings for 72.608% (10046 / 13836) of all text


In [18]:
test_text_vocab = get_vocab(df_test['text'].values)
test_reply_vocab = get_vocab(df_test['reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
unknown_text = check_coverage(test_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 17338
test reply unique vocab count is: 5187


Found embeddings for 39.826% (6905 / 17338) of vocab
Found embeddings for 78.101% (63967 / 81903) of all text



Found embeddings for 45.171% (2343 / 5187) of vocab
Found embeddings for 72.828% (9641 / 13238) of all text


### Try to convert to lower case

In [19]:
def get_vocab_lower(corpus):
    vocabulary = Counter()
    for sentance in corpus:
        for word in sentance.lower().split():
            vocabulary.update([word])
    return vocabulary

In [20]:
train_text_vocab = get_vocab_lower(df_train['text'].values)
train_reply_vocab = get_vocab_lower(df_train['reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
unknown_text_lower = check_coverage(train_text_vocab, roberta_vocab)
print()
unknown_reply_lower = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 60613
train reply unique vocab count is: 22586


Found embeddings for 19.479% (11807 / 60613) of vocab
Found embeddings for 77.394% (506905 / 654963) of all text



Found embeddings for 27.344% (6176 / 22586) of vocab
Found embeddings for 73.495% (79665 / 108395) of all text


In [21]:
dev_text_vocab = get_vocab_lower(df_dev['text'].values)
dev_reply_vocab = get_vocab_lower(df_dev['reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 15888
dev reply unique vocab count is: 4818


Found embeddings for 35.203% (5593 / 15888) of vocab
Found embeddings for 77.151% (64014 / 82972) of all text



Found embeddings for 42.777% (2061 / 4818) of vocab
Found embeddings for 73.143% (10120 / 13836) of all text


In [22]:
test_text_vocab = get_vocab_lower(df_test['text'].values)
test_reply_vocab = get_vocab_lower(df_test['reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
unknown_text = check_coverage(test_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 15473
test reply unique vocab count is: 4674


Found embeddings for 36.418% (5635 / 15473) of vocab
Found embeddings for 77.830% (63745 / 81903) of all text



Found embeddings for 42.576% (1990 / 4674) of vocab
Found embeddings for 73.236% (9695 / 13238) of all text


### Show some unknown tokens!

In [23]:
unknown_text

{'@youngdeji_': 1,
 'uzi': 2,
 'carti': 4,
 'monday': 3,
 'woah': 1,
 'we’re': 27,
 'safety.': 1,
 'dababy': 4,
 'niggas': 10,
 'denny’s': 1,
 'indians': 3,
 'don’t': 126,
 'cricket.': 1,
 'would’ve': 3,
 'out.': 21,
 'zaira': 1,
 'wasim': 1,
 'hardwork': 1,
 'screen.': 1,
 'sick.': 4,
 '@madisonbeer': 1,
 'i’ve': 57,
 '“as': 2,
 'please”': 1,
 'wtf': 8,
 '@lupeloops': 1,
 '😓': 3,
 "haven't": 7,
 'ops,': 1,
 "ain't": 6,
 'disappointments....': 1,
 'work.': 6,
 'cried.': 1,
 'settings,': 1,
 'home,': 3,
 'it.': 94,
 'nightmare.': 3,
 'spamming': 4,
 'mad.': 4,
 'couldn’t': 10,
 'back.': 14,
 'replying': 5,
 '“who': 4,
 'are?!”': 3,
 'up.': 25,
 '😰': 4,
 'understands.': 1,
 'prophet.': 1,
 'doing?': 4,
 'i’m': 263,
 'overreacting,': 1,
 'obsessing,': 1,
 'worrying,': 1,
 'disassociating.': 1,
 'here!': 2,
 'covid-19': 34,
 'vaccine?': 11,
 'myrtle': 1,
 'am.': 3,
 'spiraling.': 1,
 'grudges.': 1,
 'so,': 7,
 'mind,': 2,
 "who's": 9,
 'me?': 11,
 'oldrich': 6,
 'people,': 13,
 'judgement,

## Clean weird punctuations

#### No significantly improve

In [24]:
def clean_weird(text):
    specials = ["’", "‘", "´", "`"]
    text = text.replace("’", "'")
    text = text.replace("‘", "'")
    text = text.replace("´", "'")
    text = text.replace("`", "'")
    return text

In [25]:
df_train['text'] = df_train.text.apply(clean_weird)
df_train['reply'] = df_train.reply.apply(clean_weird)

In [26]:
df_dev['text'] = df_dev.text.apply(clean_weird)
df_dev['reply'] = df_dev.reply.apply(clean_weird)

In [27]:
df_test['text'] = df_test.text.apply(clean_weird)
df_test['reply'] = df_test.reply.apply(clean_weird)

In [28]:
train_text_vocab = get_vocab(df_train['text'].values)
train_reply_vocab = get_vocab(df_train['reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
unknown_text = check_coverage(train_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 68710
train reply unique vocab count is: 25436


Found embeddings for 22.908% (15740 / 68710) of vocab
Found embeddings for 77.684% (508799 / 654963) of all text



Found embeddings for 30.441% (7743 / 25436) of vocab
Found embeddings for 73.166% (79308 / 108395) of all text


In [29]:
dev_text_vocab = get_vocab(df_dev['text'].values)
dev_reply_vocab = get_vocab(df_dev['reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 17597
dev reply unique vocab count is: 5322


Found embeddings for 39.257% (6908 / 17597) of vocab
Found embeddings for 77.473% (64281 / 82972) of all text



Found embeddings for 45.584% (2426 / 5322) of vocab
Found embeddings for 72.615% (10047 / 13836) of all text


In [30]:
test_text_vocab = get_vocab(df_test['text'].values)
test_reply_vocab = get_vocab(df_test['reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
unknown_text = check_coverage(test_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 17246
test reply unique vocab count is: 5152


Found embeddings for 40.044% (6906 / 17246) of vocab
Found embeddings for 78.102% (63968 / 81903) of all text



Found embeddings for 45.477% (2343 / 5152) of vocab
Found embeddings for 72.828% (9641 / 13238) of all text


## Transform apostrophes

In [31]:
apostrophes = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [32]:
def change_apostrophes(text):
    # Replace apostrophes to original term
    for key in apostrophes.keys():
        text = text.replace(key, apostrophes[key])
    return text

In [33]:
df_train['text'] = df_train.text.apply(change_apostrophes)
df_train['reply'] = df_train.reply.apply(change_apostrophes)

In [34]:
df_dev['text'] = df_dev.text.apply(change_apostrophes)
df_dev['reply'] = df_dev.reply.apply(change_apostrophes)

In [35]:
df_test['text'] = df_test.text.apply(change_apostrophes)
df_test['reply'] = df_test.reply.apply(change_apostrophes)

In [36]:
train_text_vocab = get_vocab(df_train['text'].values)
train_reply_vocab = get_vocab(df_train['reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
unknown_text = check_coverage(train_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 68558
train reply unique vocab count is: 25352


Found embeddings for 22.959% (15740 / 68558) of vocab
Found embeddings for 80.319% (537529 / 669242) of all text



Found embeddings for 30.546% (7744 / 25352) of vocab
Found embeddings for 75.973% (84170 / 110789) of all text


In [37]:
dev_text_vocab = get_vocab(df_dev['text'].values)
dev_reply_vocab = get_vocab(df_dev['reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 17524
dev reply unique vocab count is: 5273


Found embeddings for 39.432% (6910 / 17524) of vocab
Found embeddings for 80.157% (67975 / 84802) of all text



Found embeddings for 46.008% (2426 / 5273) of vocab
Found embeddings for 75.556% (10692 / 14151) of all text


In [38]:
test_text_vocab = get_vocab(df_test['text'].values)
test_reply_vocab = get_vocab(df_test['reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
unknown_text = check_coverage(test_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 17171
test reply unique vocab count is: 5108


Found embeddings for 40.225% (6907 / 17171) of vocab
Found embeddings for 80.592% (67376 / 83601) of all text



Found embeddings for 45.869% (2343 / 5108) of vocab
Found embeddings for 75.643% (10236 / 13532) of all text


In [39]:
unknown_text

{'@Youngdeji_': 1,
 'uzi': 2,
 'carti': 3,
 'woah': 1,
 'safety.': 1,
 'dababy': 3,
 'niggas': 6,
 "Denny's": 1,
 'cricket.': 1,
 'out.': 21,
 'Zaira': 1,
 'Wasim': 1,
 'hardwork': 1,
 'screen.': 1,
 'sick.': 4,
 '@madisonbeer': 1,
 '“as': 2,
 'please”': 1,
 'wtf': 5,
 '@LupeLoops': 1,
 '😓': 3,
 'ops,': 1,
 'disappointments....': 1,
 'work.': 6,
 'cried.': 1,
 'settings,': 1,
 'home,': 3,
 'it.': 93,
 'nightmare.': 3,
 'spamming': 4,
 'mad.': 4,
 'back.': 13,
 'replying': 5,
 '“Who': 3,
 'are?!”': 3,
 'PHYSICALLY': 3,
 'up.': 25,
 '😰': 4,
 'understands.': 1,
 'prophet.': 1,
 'Doing?': 1,
 'overreacting,': 1,
 'obsessing,': 1,
 'worrying,': 1,
 'disassociating.': 1,
 'here!': 2,
 'Covid-19': 15,
 'Vaccine?': 11,
 'myrtle': 1,
 'am.': 3,
 'spiraling.': 1,
 'STRATEGICALLY': 1,
 'DYNAMICALLY': 1,
 'grudges.': 1,
 'So,': 5,
 'mind,': 2,
 'me?': 9,
 'OldRich': 6,
 'people,': 12,
 'judgement,': 6,
 'kindness,': 8,
 'HUNGRY': 6,
 '(now': 6,
 '“teammates”)!': 6,
 'hyperventilate': 1,
 'bag.': 1

## Check punctuations that roberta unknown 

In [40]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def unknown_punctuation(roberta_vocab):
    unknown = ''
    for char in punct:
        if char not in list(roberta_vocab.keys()):
            unknown += char
            unknown += ' '
    return unknown

In [41]:
print("Roberta unknown: ")
print(unknown_punctuation(roberta_vocab))

Roberta unknown: 
“ ” ’ ∞ θ α • − β ∅ π ‘ ₹ € ™ √ — – 


### Mapping unknown to known punctuations

In [42]:
def change_punc(text):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])
    for p in punct:
        text = text.replace(p, ' {} '.format(p))
    return text

In [43]:
df_train['map_punc_text'] = df_train.text.apply(change_punc)
df_train['map_punc_reply'] = df_train.reply.apply(change_punc)

In [44]:
df_dev['map_punc_text'] = df_dev.text.apply(change_punc)
df_dev['map_punc_reply'] = df_dev.reply.apply(change_punc)

In [45]:
df_test['map_punc_text'] = df_test.text.apply(change_punc)
df_test['map_punc_reply'] = df_test.reply.apply(change_punc)

In [46]:
train_text_vocab = get_vocab(df_train['map_punc_text'].values)
train_reply_vocab = get_vocab(df_train['map_punc_reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
print()

unknown_text = check_coverage(train_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 46129
train reply unique vocab count is: 18569



Found embeddings for 38.009% (17533 / 46129) of vocab
Found embeddings for 92.560% (743996 / 803801) of all text



Found embeddings for 47.709% (8859 / 18569) of vocab
Found embeddings for 89.777% (124148 / 138285) of all text


In [47]:
dev_text_vocab = get_vocab(df_dev['map_punc_text'].values)
dev_reply_vocab = get_vocab(df_dev['map_punc_reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 13357
dev reply unique vocab count is: 4473


Found embeddings for 59.886% (7999 / 13357) of vocab
Found embeddings for 92.546% (94356 / 101956) of all text



Found embeddings for 63.604% (2845 / 4473) of vocab
Found embeddings for 89.144% (15635 / 17539) of all text


In [48]:
test_text_vocab = get_vocab(df_test['map_punc_text'].values)
test_reply_vocab = get_vocab(df_test['map_punc_reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
unknown_text = check_coverage(test_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 13174
test reply unique vocab count is: 4263


Found embeddings for 60.710% (7998 / 13174) of vocab
Found embeddings for 92.731% (92696 / 99962) of all text



Found embeddings for 65.142% (2777 / 4263) of vocab
Found embeddings for 89.830% (15236 / 16961) of all text


In [49]:
sorted(unknown_text.items(), key=lambda d: d[1], reverse=True) 

[('coronavirus', 43),
 ('COVID', 38),
 ('Covid', 32),
 ('COVID19', 31),
 ('pandemic', 27),
 ('BailoutHumansNow', 24),
 ('dependable', 24),
 ('😂', 24),
 ('ScottyFromMarketing', 23),
 ('❤️', 23),
 ('🤔', 23),
 ('\U0001f97a', 19),
 ('😭', 19),
 ('seanhannity', 19),
 ('2nd', 19),
 ('WritingCommunity', 19),
 ('😊', 18),
 ('gifs', 17),
 ('AGAIN', 17),
 ('😔', 16),
 ('OPENING', 16),
 ('distancing', 15),
 ('LIBERATE', 14),
 ('lmao', 14),
 ('yall', 13),
 ('corona', 13),
 ('6pm', 13),
 ('🙃', 13),
 ('Vaccine', 12),
 ('questions…', 12),
 ('covid', 12),
 ('VIRGINIA', 11),
 ('DMs', 11),
 ('\U0001f974', 11),
 ('Ventilators', 11),
 ('😳', 11),
 ('1st', 11),
 ('ventilators', 10),
 ('H1N1', 10),
 ('Swine', 10),
 ('Polling', 10),
 ('cuz', 10),
 ('😂😂😂', 10),
 ('reopening', 10),
 ('BREAKING', 10),
 ('5k', 10),
 ('👀', 9),
 ('😭😭', 9),
 ('3rd', 9),
 ('Idk', 9),
 ('🙄', 9),
 ('ppl', 9),
 ('pls', 9),
 ('idk', 8),
 ('11am', 8),
 ('nigga', 8),
 ('7th', 8),
 ('Frequently', 8),
 ('→', 8),
 ('💕', 8),
 ('Coronavirus', 8),


### Transform more words

In [50]:
more_apostrophes = {'cannot': "can not", 'gonna': "go to", 'wanna': "want to", 'coronavirus': "COVID", 'wanted': "want", 'weeks': "week", 'feeling': "feel", 'says': "say", 'yourself': "your self", 'saying': "say", 'says': "say", 'GIF': "gif", 'waiting': "wait", 'Covid': "COVID", 'hugs': "hug", 'gave': "give", 'COVID19': "COVID", 'installing': "install", 'wants': "want", 'knows': "know", 'describes': "describe", 'following': "follow", 'asked': "ask", 'amazing': "amaze", 'finally': "final", 'minutes': "minute", 'died': "die", 'tired': "tire", 'quickly': "quick", 'gotta': "go to", 'deaths': "death", 'means': "mean", 'took': "take", 'feels': "feel", 'fans': "fan", 'numbers': "number", 'lives': "live", 'safely': "safe", 'tried': "try", 'businesses': "business", '2nd': "second", 'decided': "decide", '3rd': "third", 'hates': "hate", 'dont': "do not", 'lonely': "lone", 'totally': "total", 'excited': "excite", 'BREAKING': "break", 'gifs': "gif", 'goes': "go", 'thoughts': "thought", 'campaigning': "campaign", 'immediately': "immediate", 'teammates': "team mate", 'knew': "know", 'politicians': "politician", 'distancing': "distance", 'reopening': "reopen", 'pls': "please", 'AGAIN': "again", 'tears': "tear", 'supposed': "suppose", 'loved': "love", 'ppl': "people", 'drinking': "drink", 'Guidelines': "guide line", 'losing': "lose", 'Conference': "conference", 'officially': "official", 'OPENING': "open", 'buying': "buy", 'Gif': "gif", 'looks': "look", 'bought': "buy", 'likes': "like", 'truely': "true", 'happened': "happen", 'putting': "put", 'families': "family", 'moved': "move", 'Raise': "raise", 'helped': "help", 'vibes': "vibe", 'voting': "vote", 'showed': "show", 'Instagram': "instagram", 'spent': "spend", 'watched': "watch", 'kinda': "kind of", 'Governor': "governor", 'Coronavirus': "COVID", 'lmao': "laugh", 'seems': "seem", 'staying': "stay", 'listening': "listen", 'accounts': "account"}
def change_punc(text):
    for key in more_apostrophes.keys():
        text = text.replace(key, more_apostrophes[key])
    return text

In [51]:
df_train['map_more_punc_text'] = df_train.map_punc_text.apply(change_punc)
df_train['map_more_punc_reply'] = df_train.map_punc_reply.apply(change_punc)

In [52]:
df_dev['map_more_punc_text'] = df_dev.map_punc_text.apply(change_punc)
df_dev['map_more_punc_reply'] = df_dev.map_punc_reply.apply(change_punc)

In [53]:
df_test['map_more_punc_text'] = df_test.map_punc_text.apply(change_punc)
df_test['map_more_punc_reply'] = df_test.map_punc_reply.apply(change_punc)

In [54]:
train_text_vocab = get_vocab(df_train['map_more_punc_text'].values)
train_reply_vocab = get_vocab(df_train['map_more_punc_reply'].values)
print("train text unique vocab count is: {}".format(len(train_text_vocab)))
print("train reply unique vocab count is: {}".format(len(train_reply_vocab)))
print()

unknown_text = check_coverage(train_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(train_reply_vocab, roberta_vocab)

train text unique vocab count is: 46010
train reply unique vocab count is: 18481



Found embeddings for 37.846% (17413 / 46010) of vocab
Found embeddings for 92.670% (747104 / 806199) of all text



Found embeddings for 47.443% (8768 / 18481) of vocab
Found embeddings for 89.807% (124583 / 138723) of all text


In [55]:
dev_text_vocab = get_vocab(df_dev['map_more_punc_text'].values)
dev_reply_vocab = get_vocab(df_dev['map_more_punc_reply'].values)
print("dev text unique vocab count is: {}".format(len(dev_text_vocab)))
print("dev reply unique vocab count is: {}".format(len(dev_reply_vocab)))
unknown_text = check_coverage(dev_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(dev_reply_vocab, roberta_vocab)

dev text unique vocab count is: 13257
dev reply unique vocab count is: 4415


Found embeddings for 59.644% (7907 / 13257) of vocab
Found embeddings for 92.660% (94744 / 102249) of all text



Found embeddings for 63.284% (2794 / 4415) of vocab
Found embeddings for 89.189% (15699 / 17602) of all text


In [56]:
test_text_vocab = get_vocab(df_test['map_more_punc_text'].values)
test_reply_vocab = get_vocab(df_test['map_more_punc_reply'].values)
print("test text unique vocab count is: {}".format(len(test_text_vocab)))
print("test reply unique vocab count is: {}".format(len(test_reply_vocab)))
unknown_text = check_coverage(test_text_vocab, roberta_vocab)
print()
unknown_reply = check_coverage(test_reply_vocab, roberta_vocab)

test text unique vocab count is: 13076
test reply unique vocab count is: 4209


Found embeddings for 60.477% (7908 / 13076) of vocab
Found embeddings for 92.838% (93088 / 100269) of all text



Found embeddings for 64.837% (2729 / 4209) of vocab
Found embeddings for 89.866% (15288 / 17012) of all text


In [57]:
sorted(unknown_text.items(), key=lambda d: d[1], reverse=True) 

[('COVID', 158),
 ('pandemic', 27),
 ('BailoutHumansNow', 24),
 ('dependable', 24),
 ('😂', 24),
 ('ScottyFromMarketing', 23),
 ('❤️', 23),
 ('🤔', 23),
 ('amaze', 21),
 ('\U0001f97a', 19),
 ('😭', 19),
 ('seanhannity', 19),
 ('WritingCommunity', 19),
 ('😊', 18),
 ('excite', 17),
 ('😔', 16),
 ('LIBERATE', 14),
 ('yall', 13),
 ('corona', 13),
 ('6pm', 13),
 ('🙃', 13),
 ('Vaccine', 12),
 ('questions…', 12),
 ('covid', 12),
 ('VIRGINIA', 11),
 ('DMs', 11),
 ('\U0001f974', 11),
 ('Ventilators', 11),
 ('😳', 11),
 ('1st', 11),
 ('ventilators', 10),
 ('H1N1', 10),
 ('Swine', 10),
 ('Polling', 10),
 ('cuz', 10),
 ('😂😂😂', 10),
 ('5k', 10),
 ('👀', 9),
 ('😭😭', 9),
 ('Idk', 9),
 ('🙄', 9),
 ('idk', 8),
 ('11am', 8),
 ('nigga', 8),
 ('7th', 8),
 ('Frequently', 8),
 ('instagram', 8),
 ('→', 8),
 ('💕', 8),
 ('✅', 8),
 ('🤝', 8),
 ('FollowFriday', 8),
 ('OldRich', 7),
 ('Lebron', 7),
 ('Genuinely', 7),
 ('Rashford', 7),
 ('RETWEET', 7),
 ('TESTING', 7),
 ('QUARANTINE', 7),
 ('\U0001f970', 7),
 ('🔑', 7),
 (

## Store preprocessed results

In [58]:
df_train.head()

,idx,text,reply,categories,mp4,map_punc_text,map_punc_reply,map_more_punc_text,map_more_punc_reply
0,0,we can all agree that any song by Niall Horan.,oui oui,[yes],6dc39e96b11275f064fdaed88273b45e.mp4,we can all agree that any song by Niall Horan .,oui oui,we can all agree that any song by Niall Horan .,oui oui
1,1,Will you be installing #ScottyFromMarketing's ...,,[no],cfff051f05d8d3b7136c7d58ea6ad55f.mp4,Will you be installing # ScottyFromMarketing ...,,Will you be install # ScottyFromMarketing ' ...,
2,2,Growing up my mum would call me a Nigga despit...,And he joins in??? Pour some hot grits on em,[smh],bf39e7bd9ad24354ce3ba6822b0104af.mp4,Growing up my mum would call me a Nigga despit...,And he joins in ? ? ? Pour some hot grits o...,Growing up my mum would call me a Nigga despit...,And he joins in ? ? ? Pour some hot grits o...
3,3,Rest your head on my chest when the world feel...,😂😂😂😂😂,[wink],173a707a04c277354a2f23cf01d6151e.mp4,Rest your head on my chest when the world feel...,😂😂😂😂😂,Rest your head on my chest when the world feel...,😂😂😂😂😂
4,4,Imagine Will Hernandez and Wills both doing a ...,,[yes],aab6d6bfb0c1382269ddba9b71cc8b7a.mp4,Imagine Will Hernandez and Wills both doing a ...,,Imagine Will Hernandez and Wills both doing a ...,


In [59]:
df_dev.head()

,idx,text,reply,map_punc_text,map_punc_reply,map_more_punc_text,map_more_punc_reply
0,32000,"Drop your cash app, use hashtag #BailoutHumansNow",$tyratomaro #BailoutHumans,"Drop your cash app , use hashtag # BailoutHu...",$ tyratomaro # BailoutHumans,"Drop your cash app , use hashtag # BailoutHu...",$ tyratomaro # BailoutHumans
1,32001,After interviewing with a few incredible peopl...,CONGRATS!!!!!,After interviewing with a few incredible peopl...,CONGRATS ! ! ! ! !,After interviewing with a few incredible peopl...,CONGRATS ! ! ! ! !
2,32002,I know GTC festival not happening next month b...,,I know GTC festival not happening next month b...,,I know GTC festival not happening next month b...,
3,32003,"Lordy, my daughter just said, “I wonder how th...",,"Lordy , my daughter just said , "" I wonde...",,"Lordy , my daughter just said , "" I wonde...",
4,32004,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...,THE UNEMPLOYMENT CLAIM SYSTEM SUCKS SO MUCH DICK,Watching everyone else get their weekly unempl...


In [60]:
df_test.head()

,idx,text,reply,map_punc_text,map_punc_reply,map_more_punc_text,map_more_punc_reply
0,36000,@Youngdeji_ I think if uzi and carti dropping ...,,@ Youngdeji - I think if uzi and carti dro...,,@ Youngdeji - I think if uzi and carti dro...,
1,36001,For the third year in a row we are discussing ...,,For the third year in a row we are discussing ...,,For the third year in a row we are discussing ...,
2,36002,dababy album sounds like it was made for nigga...,That's why you bought it.,dababy album sounds like it was made for nigga...,That ' s why you bought it .,dababy album sounds like it was made for nigga...,That ' s why you buy it .
3,36003,Majority of Indians do not watch any sport oth...,@ZairaWasimmm got a great story because of the...,Majority of Indians do not watch any sport oth...,@ ZairaWasimmm got a great story because of t...,Majority of Indians do not watch any sport oth...,@ ZairaWasimmm got a great story because of t...
4,36004,everybody is just now listening to @madisonbee...,,everybody is just now listening to @ madisonb...,,everybody is just now listen to @ madisonbeer...,


## Output preprocessed to json

In [61]:
df_preprocessed = df_train[['idx', 'map_more_punc_text', 'map_more_punc_reply', 'categories']].copy()
df_preprocessed.columns = ['idx', 'text', 'reply', 'categories']
df_preprocessed.to_json('./preprocessed/preprocess_train.json', orient='records', lines=True)

In [62]:
df_preprocessed_dev = df_dev[['idx', 'map_more_punc_text', 'map_more_punc_reply']].copy()
df_preprocessed_dev.columns = ['idx', 'text', 'reply']
df_preprocessed_dev.to_json('./preprocessed/preprocess_dev.json', orient='records', lines=True)

In [63]:
df_preprocessed_test = df_test[['idx', 'map_more_punc_text', 'map_more_punc_reply']].copy()
df_preprocessed_test.columns = ['idx', 'text', 'reply']
df_preprocessed_test.to_json('./preprocessed/preprocess_test.json', orient='records', lines=True)